This script is developed to get sensor data for the finished product flow out of Evap 3, and transform this information, for example by integration.

The output from Evap 3 is finished product and there are a variety of potentially useful sensors that could aid investigations targeted at correlating extraction process changes with finished product changes.

It is easy enough to get the raw data series from the data warehouse, but since we want to integrate some of sensor signals (like mass flow in kg/hr into a cumulative mass), we're well advised to make use of Python's rich math libraries, rather than trying some approximate solution with JMP.

What do we want from this script:
* Pull individual data series from GCP as 1-minute averages
    * Mass flow (kg/hr) out of Evap 3: <code>tagname = 'FI323113_EvaporatorThreeDischarge' </code> in <code>akbm-houston-prod.houston_data.sensor_data_scada</code>
    * Mass flow (lb/hr) out of Evap 3: <code>tagname = FIT_323113</code> in <code>akbm-houston-prod.houston_data.sensor_data_sulzer2</code>
    * Phospholipid concentration in mass flow out of Evap 3: <code>tagname = Evaporator 3 Discharge (AE3800A).Phospholipids.Value</code> in <code>akbm-houston-prod.houston_data.sensor_data_ftnir</code>
* Add two new columns showing the mass flow of total PL from Evap 3. This is the product between mass flow and PL percentage (formula: mass flow/100 * PL)
* Integrate (as in "calculate the area under the curve) the signals over defined time intervals (e.g. integrate output mass from Evap 3 for every 1h or every shift, and so on). Variables for pulling data:
    * Start time (06:00 AM & 06:00 PM are local CDT start times for day & night shift in Houston respectively)
    * Interval for integration (e.g. 12 hours to get integrated results for day & night shifts)
    * Granularity (SQL: SECOND, MINUTE, HOUR ...). However, 1-minute granularity is possibly not an issue considering the size of the dataset.
* Give a summary table for the integration results.
    * Use different integration methods for cross checking:
        * Trapezoidal rule
        * Simpson's rule
        * Rieman sum
        * Aggregation (OK, this is not integration, but we want it for comparison)
 * Give output as csv with clearly labelled column headers

The below script gives the following results of the specified intervals (INTERVAL_HOURS), starting with with START_TIME:
* Total mass in kg
* Total mass in lb
* PL mass in kg
* PL mass in lb

---

**IMPORTANT(?)**<br>
When testing the below script and comparing it to a manual point & click - based procedure using JMP, deviations were observed for the aggregated data. 

To reproduce this test use the following parameters:<br>
* <code>START_TIME      = datetime(2025, 5, 10, 0, 0, 0)</code><br>
* <code>END_TIME        = datetime(2025, 6, 2, 0, 0, 0)<code>
* <code>INTERVAL_HOURS  = 12</code>
* use 1-minute granularity
* use the JSL code block shown after the Python code block right below to get JMP-based dataset

On 02.Jun.'25 at around 16:30 local time in Oslo, the joined data sets containing aggregated data had 46 rows. Comparing the 12h aggregates from point & click via JMP to the Python script below, the differences for 11 of the rows were much bigger than zero. However, the relative difference ranged only from approx. 0,1-0,4% rel., which is why this isn't a high priority issue for now. These are the timestamps for the rows where deviations were found:

11.05.2025 00:00<br>
12.05.2025 12:00<br>
23.05.2025 00:00<br>
24.05.2025 12:00<br>
25.05.2025 00:00<br>
27.05.2025 12:00<br>
28.05.2025 12:00<br>
30.05.2025 00:00<br>
30.05.2025 12:00<br>
31.05.2025 00:00<br>
01.06.2025 12:00<br>

Each timestamp is the start of a 12-h interval


---

### The below code is based on 1-minute averages of the tag data

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.integrate import simpson
from typing import List, Tuple

# ─── USER‐DEFINED PARAMETERS ────────────────────────────────────────────────────
# 1) Time window for data pull (UTC)
START_TIME      = datetime(2025, 5, 10, 11, 0, 0)   # SQL‐query start AND first integration interval start, (yyyy, m, d, hr, min, sec)
END_TIME        = datetime(2025, 6, 23, 23, 0, 0)   # SQL‐query end (and integration cutoff)
# remember, time in data warehouse is UTC time

# 2) Integration interval length (hours)
INTERVAL_HOURS  = 12                          

# 3) BigQuery table & tag names
SCADA_TABLE     = "akbm-houston-prod.houston_data.sensor_data_scada"
TAG_MF_KGHR     = "FI323113_EvaporatorThreeDischarge"

SULZER_TABLE    = "akbm-houston-prod.houston_data.sensor_data_sulzer2"
TAG_MF_LBHR     = "FIT_323113"

FTNIR_TABLE     = "akbm-houston-prod.houston_data.sensor_data_ftnir"
TAG_PL_PCT      = "Evaporator 3 Discharge (AE3800A).Phospholipids.Value"

# 4) CSV output filename
OUT_CSV         = "evap3_integration_summary.csv"

# ─── 1) CONNECT TO BigQuery via ODBC ──────────────────────────────────────────
dsn = 'bq64_system'
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)

# ─── 2) FUNCTION TO PULL A 1-MINUTE‐AVERAGED SERIES ───────────────────────────
def fetch_1min_series(
    conn: pyodbc.Connection,
    dataset_table: str,
    tagname: str,
    start_time: datetime,
    end_time: datetime
) -> pd.DataFrame:
    """
    Queries BigQuery to pull a single tag as a 1-minute‐averaged time series
    between start_time and end_time (inclusive). Returns a DataFrame with:
        ┌───────────────────┬──────────┐
        │     timestamp     │   value  │
        ├───────────────────┼──────────┤
        │ 2025-06-01 00:00  │ 123.456  │
        │ 2025-06-01 00:01  │ 120.789  │
        │   ...             │   ...    │
        └───────────────────┴──────────┘
    Assumes each table has columns: `timestamp` (TIMESTAMP), `value`, and `tagname`.
    """

    t0 = start_time.strftime("%Y-%m-%d %H:%M:%S")
    t1 = end_time.strftime("%Y-%m-%d %H:%M:%S")

    sql = f"""
    SELECT
      TIMESTAMP_TRUNC(time, MINUTE) AS ts_min,
      AVG(value) AS avg_value
    FROM `{dataset_table}`
    WHERE tagname = '{tagname}'
      AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
    GROUP BY ts_min
    ORDER BY ts_min
    """

    df = pd.read_sql(sql, conn)
    df = df.rename(columns={"ts_min": "timestamp", "avg_value": "value"})
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp").sort_index()
    return df

# ─── 3) PULL THE THREE SERIES AND MERGE THEM ─────────────────────────────────
df_kg  = fetch_1min_series(conn, SCADA_TABLE, TAG_MF_KGHR, START_TIME, END_TIME)
df_lb  = fetch_1min_series(conn, SULZER_TABLE, TAG_MF_LBHR, START_TIME, END_TIME)
df_plp = fetch_1min_series(conn, FTNIR_TABLE, TAG_PL_PCT,  START_TIME, END_TIME)

df_kg  = df_kg.rename(columns={"value": "mass_flow_kg_hr"})
df_lb  = df_lb.rename(columns={"value": "mass_flow_lb_hr"})
df_plp = df_plp.rename(columns={"value": "pl_pct"})

df_all = df_kg.join(df_lb, how="inner").join(df_plp, how="inner")
# Now df_all has columns: mass_flow_kg_hr, mass_flow_lb_hr, pl_pct

# ─── 4) COMPUTE “TOTAL PL MASS” COLUMNS (kg/hr & lb/hr) ──────────────────────
df_all["pl_mass_kg_hr"] = df_all["mass_flow_kg_hr"] * (df_all["pl_pct"] / 100.0)
df_all["pl_mass_lb_hr"] = df_all["mass_flow_lb_hr"] * (df_all["pl_pct"] / 100.0)
# Now df_all columns: mass_flow_kg_hr, mass_flow_lb_hr, pl_pct, pl_mass_kg_hr, pl_mass_lb_hr

# ─── 5) BUILD INTEGRATION INTERVALS FROM SINGLE START + DURATION ──────────────
def build_intervals_from_start(
    start: datetime,
    end: datetime,
    interval_hours: int
) -> List[Tuple[datetime, datetime]]:
    """
    Splits [start, end) into successive chunks of length `interval_hours`.
    Each interval_i is [start + i*interval_hours, start + (i+1)*interval_hours),
    with the final interval clipped at `end`.
    """
    intervals: List[Tuple[datetime, datetime]] = []
    cursor = start
    delta = timedelta(hours=interval_hours)

    while cursor < end:
        next_point = cursor + delta
        s = cursor
        e = min(next_point, end)
        intervals.append((s, e))
        cursor = next_point

    return intervals

interval_list = build_intervals_from_start(START_TIME, END_TIME, INTERVAL_HOURS)

# ─── 6) FUNCTION TO COMPUTE INTEGRALS PER INTERVAL ────────────────────────────
def integrate_interval(
    df: pd.DataFrame,
    col: str,
    start: datetime,
    end: datetime
) -> Tuple[float, float, float, float]:
    """
    Given a DataFrame `df` indexed by timestamp (1-minute frequency),
    and a column name `col` (e.g. 'mass_flow_kg_hr'),
    plus an interval [start, end), compute:
      1) trapezoidal rule
      2) Simpson’s rule
      3) Left‐Riemann sum
      4) aggregation = sum of values

    All flows are in “per hour” units (e.g. kg/hr). Integrating over hours
    yields “kg” (or “lb” if original was lb/hr).
    Returns (trapz, simpson, riemann, agg_sum).
    """
    df_int = df.loc[start : end - timedelta(minutes=1), col].dropna()
    if df_int.empty:
        return (0.0, 0.0, 0.0, 0.0)

    # Convert timestamps to hours since epoch
    t_secs = (df_int.index.view(np.int64) // 10**9).astype(float)
    t_hours = t_secs / 3600.0
    y = df_int.values

    # A) Trapezoidal
    trapz_val = np.trapz(y, x=t_hours)

    # B) Simpson’s rule (needs ≥3 points; drop last if even count)
    if len(y) >= 3:
        if len(y) % 2 == 0:
            t_sim = t_hours[:-1]
            y_sim = y[:-1]
        else:
            t_sim = t_hours
            y_sim = y
        simpson_val = simpson(y_sim, x=t_sim)
    else:
        simpson_val = np.nan

    # C) Left‐Riemann sum: sum(y[i] * (t[i+1] − t[i])) for i=0..N-2
    if len(y) >= 2:
        riemann_val = np.sum(y[:-1] * (t_hours[1:] - t_hours[:-1]))
    else:
        riemann_val = 0.0

    # D) Aggregation (sum of raw values)
    agg_sum_val = np.sum(y) * (1.0 / 60.0)

    return (trapz_val, simpson_val, riemann_val, agg_sum_val)

# ─── 7) LOOP OVER INTERVALS AND BUILD SUMMARY TABLE ───────────────────────────
records = []
for (t0, t1) in interval_list:
    trapz_kg, simps_kg, riem_kg, agg_kg       = integrate_interval(df_all, "mass_flow_kg_hr", t0, t1)
    trapz_plkg, simps_plkg, riem_plkg, agg_plkg = integrate_interval(df_all, "pl_mass_kg_hr",   t0, t1)
    trapz_lb, simps_lb, riem_lb, agg_lb       = integrate_interval(df_all, "mass_flow_lb_hr", t0, t1)
    trapz_pllb, simps_pllb, riem_pllb, agg_pllb = integrate_interval(df_all, "pl_mass_lb_hr",   t0, t1)

    records.append({
        "interval_start":    t0,
        "interval_end":      t1,
        # mass_flow_kg results (kg)
        "trapz_mass_kg":     trapz_kg,
        "simpson_mass_kg":   simps_kg,
        "riemann_mass_kg":   riem_kg,
        "agg_sum_mass_kg":   agg_kg,
        # pl_mass_kg results (kg)
        "trapz_pl_kg":       trapz_plkg,
        "simpson_pl_kg":     simps_plkg,
        "riemann_pl_kg":     riem_plkg,
        "agg_sum_pl_kg":     agg_plkg,
        # mass_flow_lb results (lb)
        "trapz_mass_lb":     trapz_lb,
        "simpson_mass_lb":   simps_lb,
        "riemann_mass_lb":   riem_lb,
        "agg_sum_mass_lb":   agg_lb,
        # pl_mass_lb results (lb)
        "trapz_pl_lb":       trapz_pllb,
        "simpson_pl_lb":     simps_pllb,
        "riemann_pl_lb":     riem_pllb,
        "agg_sum_pl_lb":     agg_pllb,
    })

summary_df = pd.DataFrame.from_records(records)
summary_df["interval_start"] = pd.to_datetime(summary_df["interval_start"])
summary_df["interval_end"]   = pd.to_datetime(summary_df["interval_end"])
summary_df = summary_df.sort_values("interval_start").reset_index(drop=True)

# ─── 8) WRITE SUMMARY TABLE TO CSV ────────────────────────────────────────────
summary_df.to_csv(OUT_CSV, index=False)
print(f"Wrote summary table with {len(summary_df)} intervals to → {OUT_CSV}")


---

### The code below is based on the raw data as is in the data warehouse (i.e., 1-second resolution)
<u>**NB**</u>: The below code uses resampling with interpolation, which causes issues when production abruptly stops due to a scheduled shutdown, for example. However, comparing the output of the below script to the one using 1-minute resolution, the 12h-integration results are as good as indistinguishable.

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.integrate import simpson
from typing import List, Tuple

# ─── USER‐DEFINED PARAMETERS ────────────────────────────────────────────────────
START_TIME      = datetime(2025, 5, 10, 11, 0, 0)
END_TIME        = datetime(2025, 6, 23, 23, 0, 0)
INTERVAL_HOURS  = 12

SCADA_TABLE     = "akbm-houston-prod.houston_data.sensor_data_scada"
TAG_MF_KGHR     = "FI323113_EvaporatorThreeDischarge"

SULZER_TABLE    = "akbm-houston-prod.houston_data.sensor_data_sulzer2"
TAG_MF_LBHR     = "FIT_323113"

FTNIR_TABLE     = "akbm-houston-prod.houston_data.sensor_data_ftnir"
TAG_PL_PCT      = "Evaporator 3 Discharge (AE3800A).Phospholipids.Value"

OUT_CSV         = "evap3_integration_summary.csv"

# ─── 1) CONNECT TO BigQuery via ODBC ──────────────────────────────────────────
dsn = 'bq64_system'
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)

# ─── 2) FUNCTION TO PULL RAW SERIES AND RESAMPLE TO 1-SECOND ─────────────────
def fetch_raw_series(
    conn: pyodbc.Connection,
    dataset_table: str,
    tagname: str,
    start_time: datetime,
    end_time: datetime
) -> pd.DataFrame:
    """
    Pulls raw timestamp, value for the given tag and period,
    resamples to true 1-second frequency with linear interpolation.
    """
    t0 = start_time.strftime("%Y-%m-%d %H:%M:%S")
    t1 = end_time.strftime("%Y-%m-%d %H:%M:%S")

    sql = f"""
    SELECT
      time AS timestamp,
      value
    FROM `{dataset_table}`
    WHERE tagname = '{tagname}'
      AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
    ORDER BY timestamp
    """

    df = pd.read_sql(sql, conn)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp").sort_index()

    # Resample to 1-second frequency, interpolate missing points
    df = df.resample('1S').mean().interpolate()

    return df

# ─── 3) PULL THE THREE SERIES AND MERGE THEM ─────────────────────────────────
df_kg  = fetch_raw_series(conn, SCADA_TABLE, TAG_MF_KGHR, START_TIME, END_TIME)
df_lb  = fetch_raw_series(conn, SULZER_TABLE, TAG_MF_LBHR, START_TIME, END_TIME)
df_plp = fetch_raw_series(conn, FTNIR_TABLE, TAG_PL_PCT,  START_TIME, END_TIME)

df_kg  = df_kg.rename(columns={"value": "mass_flow_kg_hr"})
df_lb  = df_lb.rename(columns={"value": "mass_flow_lb_hr"})
df_plp = df_plp.rename(columns={"value": "pl_pct"})

df_all = df_kg.join(df_lb, how="inner").join(df_plp, how="inner")

# ─── 4) COMPUTE “TOTAL PL MASS” COLUMNS ──────────────────────────────────────
df_all["pl_mass_kg_hr"] = df_all["mass_flow_kg_hr"] * (df_all["pl_pct"] / 100.0)
df_all["pl_mass_lb_hr"] = df_all["mass_flow_lb_hr"] * (df_all["pl_pct"] / 100.0)

# ─── 5) BUILD INTEGRATION INTERVALS ──────────────────────────────────────────
def build_intervals_from_start(
    start: datetime,
    end: datetime,
    interval_hours: int
) -> List[Tuple[datetime, datetime]]:
    intervals: List[Tuple[datetime, datetime]] = []
    cursor = start
    delta = timedelta(hours=interval_hours)

    while cursor < end:
        next_point = cursor + delta
        intervals.append((cursor, min(next_point, end)))
        cursor = next_point

    return intervals

interval_list = build_intervals_from_start(START_TIME, END_TIME, INTERVAL_HOURS)

# ─── 6) INTEGRATION FUNCTION REMAINS UNCHANGED ──────────────────────────────
def integrate_interval(
    df: pd.DataFrame,
    col: str,
    start: datetime,
    end: datetime
) -> Tuple[float, float, float, float]:
    df_int = df.loc[start : end - timedelta(seconds=1), col].dropna()
    if df_int.empty:
        return (0.0, 0.0, 0.0, 0.0)

    t_secs = (df_int.index.view(np.int64) // 10**9).astype(float)
    t_hours = t_secs / 3600.0
    y = df_int.values

    trapz_val = np.trapz(y, x=t_hours)

    if len(y) >= 3:
        if len(y) % 2 == 0:
            simpson_val = simpson(y[:-1], x=t_hours[:-1])
        else:
            simpson_val = simpson(y, x=t_hours)
    else:
        simpson_val = np.nan

    if len(y) >= 2:
        riemann_val = np.sum(y[:-1] * (t_hours[1:] - t_hours[:-1]))
    else:
        riemann_val = 0.0

    agg_sum_val = np.sum(y) * (1.0 / 3600.0)

    return (trapz_val, simpson_val, riemann_val, agg_sum_val)

# ─── 7) LOOP OVER INTERVALS AND BUILD SUMMARY TABLE ─────────────────────────
records = []
for (t0, t1) in interval_list:
    trapz_kg, simps_kg, riem_kg, agg_kg         = integrate_interval(df_all, "mass_flow_kg_hr", t0, t1)
    trapz_plkg, simps_plkg, riem_plkg, agg_plkg = integrate_interval(df_all, "pl_mass_kg_hr",   t0, t1)
    trapz_lb, simps_lb, riem_lb, agg_lb         = integrate_interval(df_all, "mass_flow_lb_hr", t0, t1)
    trapz_pllb, simps_pllb, riem_pllb, agg_pllb = integrate_interval(df_all, "pl_mass_lb_hr",   t0, t1)

    records.append({
        "interval_start":    t0,
        "interval_end":      t1,
        "trapz_mass_kg":     trapz_kg,
        "simpson_mass_kg":   simps_kg,
        "riemann_mass_kg":   riem_kg,
        "agg_sum_mass_kg":   agg_kg,
        "trapz_pl_kg":       trapz_plkg,
        "simpson_pl_kg":     simps_plkg,
        "riemann_pl_kg":     riem_plkg,
        "agg_sum_pl_kg":     agg_plkg,
        "trapz_mass_lb":     trapz_lb,
        "simpson_mass_lb":   simps_lb,
        "riemann_mass_lb":   riem_lb,
        "agg_sum_mass_lb":   agg_lb,
        "trapz_pl_lb":       trapz_pllb,
        "simpson_pl_lb":     simps_pllb,
        "riemann_pl_lb":     riem_pllb,
        "agg_sum_pl_lb":     agg_pllb,
    })

summary_df = pd.DataFrame.from_records(records)
summary_df["interval_start"] = pd.to_datetime(summary_df["interval_start"])
summary_df["interval_end"]   = pd.to_datetime(summary_df["interval_end"])
summary_df = summary_df.sort_values("interval_start").reset_index(drop=True)

# ─── 8) WRITE SUMMARY TABLE TO CSV ───────────────────────────────────────────
summary_df.to_csv(OUT_CSV, index=False)
print(f"Wrote summary table with {len(summary_df)} intervals to → {OUT_CSV}")


---

### Not sure what the below was written for, but looks like it can be used for raw data investigation

In [ ]:
# this can be used for analytical purpose to get the 1'' raw data from the dwh

import pyodbc
import pandas as pd
from datetime import datetime
from typing import List

# ─── USER‐DEFINED PARAMETERS ────────────────────────────────────────────────────
# 1) Time window for data pull (UTC)
START_TIME   = datetime(2025, 6, 11, 23, 0, 0)   # yyyy, m, d, H, M, S
STOP_TIME    = datetime(2025, 6, 12, 11, 0, 0) # yyyy, m, d, H, M, S

# 2) Granularity for TIMESTAMP_TRUNC (e.g. "SECOND", "MINUTE", "HOUR")
GRANULARITY = "SECOND"

# 3) BigQuery tables & tag lists
FTNIR_TABLE     = "akbm-houston-prod.houston_data.sensor_data_ftnir"
FTNIR_TAGS      = [
    "Evaporator 3 Discharge (AE3800A).Phospholipids.Value",
    "Evaporator 3 Discharge (AE3800A).Ethanol.Value",
    "Evaporator 3 Discharge (AE3800A).OM3.Value"
]

SCADA_TABLE     = "akbm-houston-prod.houston_data.sensor_data_scada"
SCADA_TAG       = "FI323113_EvaporatorThreeDischarge"

SULZER2_TABLE   = "akbm-houston-prod.houston_data.sensor_data_sulzer2"
SULZER2_TAG     = "FIT_323113"

SULZER1_TABLE   = "akbm-houston-prod.houston_data.sensor_data_sulzer1"
SULZER1_TAGS    = [
    "AE3800B_Viscosity",
    "AE3800B_Viscosity35C"
]

# 4) Output CSV filename
OUT_CSV = "evap3_one_second_sample.csv"

# ─── 1) CONNECT TO BigQuery via ODBC ──────────────────────────────────────────
dsn = 'bq64_system'
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)

# ─── 2) BUILD AND RUN UNION ALL SQL FOR MULTIPLE TAGS ─────────────────────────
def fetch_all_tags_1sec(
    conn: pyodbc.Connection,
    start_time: datetime,
    stop_time: datetime,
    granularity: str
) -> pd.DataFrame:
    """
    Pulls 1-second truncated, per-second averages for all relevant tags across
    FTNIR, SCADA, Sulzer2, and Sulzer1 tables. Returns a DataFrame with columns:
        ┌────────────┬──────────────────────────────────────────┬──────────┐
        │   time     │                tagname                  │  value   │
        ├────────────┼──────────────────────────────────────────┼──────────┤
        │ 2025-05-10 00:00:00 │ Evaporator 3 … Phospholipids.Value   │  12.345  │
        │ 2025-05-10 00:00:00 │ Evaporator 3 … Ethanol.Value        │   0.123  │
        │       …    │                    …                     │    …     │
        └────────────┴──────────────────────────────────────────┴──────────┘
    """

    t0 = start_time.strftime("%Y-%m-%d %H:%M:%S")
    t1 = stop_time.strftime("%Y-%m-%d %H:%M:%S")

    # Build the UNION ALL SQL string
    sql_parts: List[str] = []

    # 2.1) FTNIR tags
    ft_columns = ",\n    ".join(f"'{tag}'" for tag in FTNIR_TAGS)
    sql_ftnir = f"""
SELECT
  TIMESTAMP_TRUNC(time, {granularity}) AS time,
  tagname,
  AVG(value) AS value
FROM `{FTNIR_TABLE}`
WHERE tagname IN ({ft_columns})
  AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
GROUP BY tagname, time
"""
    sql_parts.append(sql_ftnir)

    # 2.2) SCADA tag
    sql_scada = f"""
SELECT
  TIMESTAMP_TRUNC(time, {granularity}) AS time,
  tagname,
  AVG(value) AS value
FROM `{SCADA_TABLE}`
WHERE tagname = '{SCADA_TAG}'
  AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
GROUP BY tagname, time
"""
    sql_parts.append(sql_scada)

    # 2.3) Sulzer2 tag
    sql_sulzer2 = f"""
SELECT
  TIMESTAMP_TRUNC(time, {granularity}) AS time,
  tagname,
  AVG(value) AS value
FROM `{SULZER2_TABLE}`
WHERE tagname = '{SULZER2_TAG}'
  AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
GROUP BY tagname, time
"""
    sql_parts.append(sql_sulzer2)

    # 2.4) Sulzer1 tags
    s1_columns = ",\n    ".join(f"'{tag}'" for tag in SULZER1_TAGS)
    sql_sulzer1 = f"""
SELECT
  TIMESTAMP_TRUNC(time, {granularity}) AS time,
  tagname,
  AVG(value) AS value
FROM `{SULZER1_TABLE}`
WHERE tagname IN ({s1_columns})
  AND time BETWEEN TIMESTAMP('{t0}') AND TIMESTAMP('{t1}')
GROUP BY tagname, time
"""
    sql_parts.append(sql_sulzer1)

    # Combine all parts with UNION ALL and final ORDER BY
    union_sql = "\nUNION ALL\n".join(sql_parts)
    full_sql = f"""
{union_sql}
ORDER BY time, tagname
"""

    # Execute and return DataFrame
    df = pd.read_sql(full_sql, conn)
    # Ensure timestamp column is parsed as pandas datetime
    df["time"] = pd.to_datetime(df["time"])
    return df

# Fetch raw per-second averages for all tags
df_raw = fetch_all_tags_1sec(conn, START_TIME, STOP_TIME, GRANULARITY)

# ─── 3) PIVOT RAW DATA SO EACH TAG BECOMES A COLUMN ───────────────────────────
# Resulting DataFrame will have index = time, columns = tagname, values = avg_value
df_pivot = df_raw.pivot(index="time", columns="tagname", values="value")

# Optionally reorder columns in a sensible order:
desired_order = (
    FTNIR_TAGS
    + [SCADA_TAG]
    + [SULZER2_TAG]
    + SULZER1_TAGS
)
df_pivot = df_pivot.reindex(columns=desired_order)

# ─── 4) WRITE TO CSV ─────────────────────────────────────────────────────────
df_pivot.to_csv(OUT_CSV, index_label="time")
print(f"Wrote {len(df_pivot)} rows × {len(df_pivot.columns)} tags → {OUT_CSV}")
